# Contents
- [Introduction]()
- [Testset generation]()
- [Build RAG with llama-index]()
- [Tracing using Phoenix]()
- [Evaluation]()
- [Embedding analysis]()
- [Conclusion]()

TODO

- ✅ Launch Phoenix
- ✅ Index corpus
- ✅ Instrument LlamaIndex
- ✅ Run LlamaIndex application
- ✅ Look at UI, understand traces
- ❌ Export and flatten span data into a phoenix dataset
- ✅ Instrument LangChain
- ✅ Run evaluations
- ✅ Show evaluations
- ✅ Map root span IDs over Ragas evaluations, create `SpanEvaluations`, used `add_evaluations` API to attach evaluations to trace dataset and launch Phoenix with that trace dataset
- ✅ Display traces with annotated evaluations
- ❌ Export using the query DSL to get a primary dataset for visualizing embeddings. Alternatively, call `get_spans_dataframe` on trace `px.TraceDataset`. Whichever is simpler.
- ❌ Wrangle out the corpus dataset from LlamaIndex in-memory vector store
- ❌ Re-launch Phoenix with primary and corpus datasets

## Introduction

In this notebook

In [ ]:
!pip install ragas pypdf arize-phoenix llama-index pandas

In [1]:
import pandas as pd

# Display the complete contents of dataframe cells.
pd.set_option("display.max_colwidth", None)

## Synthetic Test data generation

Follow the instructions [here](https://docs.github.com/en/repositories/working-with-files/managing-large-files/installing-git-large-file-storage) to install `git-lfs`.

In [ ]:
! git lfs install
! git clone https://huggingface.co/datasets/explodinggradients/prompt-engineering-papers

In [2]:
from llama_index import SimpleDirectoryReader

dir_path = "./prompt-engineering-papers"
reader = SimpleDirectoryReader(dir_path, num_files_limit=2)
documents = reader.load_data()

In [3]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

# generator with openai models
generator = TestsetGenerator.with_openai()

# set question type distribution
distribution = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}

# generate testset
testset = generator.generate_with_llamaindex_docs(
    documents, test_size=10, distributions=distribution
)
test_df = testset.to_pandas()
test_df.head()

embedding nodes:   0%|          | 0/222 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,episode_done
0,How does the lattice width of F relate to the diameter of F(M)?,"[1miisapathofminimallength, then ∥u′−v′∥ ≤r·∥M∥\nand the claim follows from diam( F(M))≥distF(M)(u′,v′) =r. □\nRemark 3.2. LetF ⊂Zdbe a normal set. For all l∈ {−1,0,1}dandu,v∈ Fwe have\n(u−v)Tl≤ ∥u−v∥1and thus width l(F) := max {(u−v)Tl:u,v∈ F} ≤ max{∥u−v∥1:\nu,v∈ F}. Suppose that u′,v′∈ Fare such that ∥u′−v′∥1= max{∥u−v∥1:u,v∈ F}and\nletl′\ni:= sign(u′\ni−v′\ni) fori∈[d], then\n∥u′−v′∥1= (u′−v′)T·l′≤widthl′(F)≤max{∥u−v∥1:u,v∈ F}=∥u′−v′∥1.\nThelattice width ofFis width( F) := minl∈Zdwidthl(F) and thus Lemma 3.1gives\n∥M∥1·diam(F(M))≥width(F).]",The lattice width of F is related to the diameter of F(M) by the inequality ∥M∥1·diam(F(M))≥width(F).,simple,True
1,How is distillation used to improve the reasoning ability of language models?,"[\nlanguage models such as T5-XXL. The distillationis achieved by finetuning the small model on the\nchain-of-thought data (Wei et al., 2022c) generated\nby a large teacher model. Although promising per-\nformance is achieved, the improvements are likely\ntask-dependent. Further investigation on improv-\ning the reasoning ability by learning from larger\nLLMs could be an interesting direction.\n11.3 ICL Robustness\nPrevious studies have shown that ICL performance\nis extremely unstable, from random guess to SOTA,\nand can be sensitive to many factors, including\ndemonstration permutation, demonstration format,\netc. (Zhao et al., 2021; Lu et al., 2022). The robust-\nness of ICL is a critical yet challenging problem.\nHowever, most of the existing methods fall into\nthe dilemma of accuracy and robustness (Chen\net al., 2022c), or even at the cost of sacrificing\ninference efficiency. To effectively improve the\nrobustness of ICL, we need deeper analysis of the\nworking mechanism of the ICL. We believe that\nthe analysis of the robustness of the ICL from a\nmore theoretical perspective rather than an empir-\nical perspective can highlight future research on\nmore robust ICL.\n11.4 ICL Efficiency and Scalability\nICL necessitates prepending a significant number\nof demonstrations within the context. However, it\npresents two challenges: (1) the quantity of demon-\nstrations is constrained by the maximum input\nlength of LMs, which is significantly fewer com-\npared to fine-tuning (scalability); (2) as the number\nof demonstrations increases, the computation cost\nbecomes higher due to the quadratic complexity of\nattention mechanism (efficiency). Previous work in\n§5 focused on exploring how to achieve better ICL\nperformance using a limited number of demonstra-\ntions and proposed several demonstration design-\ning strategies. Scaling ICL to more demonstrations\nand improving its efficiency remains a challenging\ntask.\nRecently, some works have been proposed to ad-\ndress the issues of scalability and efficiency of ICL.\nEfforts were made to optimize prompting strate-\ngies with structured prompting (Hao et al., 2022b),\ndemonstration ensembling (Khalifa et al., 2023),\ndynamic prompting]",Distillation is used to improve the reasoning ability of language models by finetuning a small model on the chain-of-thought data generated by a large teacher model.,simple,True
2,How do Transformers utilize task recognition ability and implicit empirical risk minimization for implementing a proper function class?,"[ Trans-\nformers can implement a proper function class\nthrough implicit empirical risk minimization for\nthe demonstrations. Pan et al. (2023) decoupled the\nICL ability into task recognition ability and task\nlearning ability, and further showed how they uti-lize demonstrations. From an information-theoretic\nperspective, Hahn and Goyal (2023) showed an er-\nror bound for ICL under linguistically motivated\nassumptions to explain how next-token prediction\ncan bring about the ICL ability. Si et al. (2023)\nfound that large language models exhibit prior fea-\nture biases an

## Build RAG with llama-index

In [4]:
import phoenix as px

session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
import llama_index

llama_index.set_global_handler("arize_phoenix")

In [6]:
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.embeddings import OpenAIEmbedding
from datasets import Dataset
from tqdm.auto import tqdm


def build_query_engine(documents):
    vector_index = VectorStoreIndex.from_documents(
        documents,
        service_context=ServiceContext.from_defaults(chunk_size=512),
        embed_model=OpenAIEmbedding(),
    )

    query_engine = vector_index.as_query_engine(similarity_top_k=2)
    return query_engine


def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }


# Function to evaluate as Llama index does not support async evaluation for HFInference API
def generate_ragas_dataset(query_engine, test_df):
    test_questions = test_df["question"].values
    responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]

    dataset_dict = {
        "question": test_questions,
        "answer": [response["answer"] for response in responses],
        "contexts": [response["contexts"] for response in responses],
        "ground_truth": test_df["ground_truth"].values.tolist(),
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds

In [7]:
query_engine = build_query_engine(documents)
ragas_eval_dataset = generate_ragas_dataset(query_engine, test_df)
pd.DataFrame(ragas_eval_dataset)

  0%|          | 0/10 [00:00<?, ?it/s]

,question,answer,contexts,ground_truth
0,How does the lattice width of F relate to the diameter of F(M)?,"The lattice width of F, denoted as width(F), is defined as the minimum of the lattice widths of F in all dimensions. On the other hand, the diameter of F(M), denoted as diam(F(M)), is a lower bound on the shortest path length between any two elements in F(M). \n\nBased on the given context, it can be inferred that the lattice width of F is related to the diameter of F(M) through the inequality: \n\nwidth(F) ≤ diam(F(M)) / ∥M∥\n\nThis means that the lattice width of F is bounded above by the diameter of F(M) divided by the maximum norm of the elements in M.","[□\nRemark 3.2. LetF ⊂Zdbe a normal set. For all l∈ {−1,0,1}dandu,v∈ Fwe have\n(u−v)Tl≤ ∥u−v∥1and thus width l(F) := max {(u−v)Tl:u,v∈ F} ≤ max{∥u−v∥1:\nu,v∈ F}. Suppose that u′,v′∈ Fare such that ∥u′−v′∥1= max{∥u−v∥1:u,v∈ F}and\nletl′\ni:= sign(u′\ni−v′\ni) fori∈[d], then\n∥u′−v′∥1= (u′−v′)T·l′≤widthl′(F)≤max{∥u−v∥1:u,v∈ F}=∥u′−v′∥1.\nThelattice width ofFis width( F) := minl∈Zdwidthl(F) and thus Lemma 3.1gives\n∥M∥1·diam(F(M))≥width(F)., 3.Bounds on the diameter\nIn general knowledge of the diameter of the graph underlying a Markov chain can provide\ninformation about the mixing time. For random walks on ﬁber g raphs, the chains which we\nconsider, the underlying graph coincides with the ﬁber grap h. In this section, we determine\nlower and upper bounds on the diameter of ﬁber graphs and thei r compressed counterparts.\nFor a ﬁnite set M ⊂Zdand any norm ∥·∥onRd, let∥M∥:= max m∈M∥m∥.\nLemma 3.1. LetF ⊂ZdandM ⊂Zdbe ﬁnite sets, then\ndiam(F(M))≥1\n∥M∥·max{∥u−v∥:u,v∈ F}.\nProof.IfF(M) is not connected, then the statement holds trivially, so as sume that Mis a\nMarkov basis for F. Letu′,v′∈ Fsuch that ∥u′−v′∥= max{∥u−v∥:u,v∈ F}and let\nm1,...,m r∈ Msothatu′=v′+∑r\ni=1miisapathofminimallength, then ∥u′−v′∥ ≤r·∥M∥\nand the claim follows from diam( F(M))≥distF(M)(u′,v′) =r. □\nRemark 3.2. LetF ⊂Zdbe a normal set. For all l∈ {−1,0,1}dandu,v∈ Fwe have\n(u−v)Tl≤ ∥u−v∥1and thus width l(F) := max {(u−v)Tl:u,v∈ F} ≤ max{∥u−v∥1:\nu,v∈ F}.]",The lattice width of F is related to the diameter of F(M) by the inequality ∥M∥1·diam(F(M))≥width(F).
1,How is distillation used to improve the reasoning ability of language models?,"Distillation is used to improve the reasoning ability of language models by transferring the reasoning ability from larger models to smaller models. This is achieved by finetuning the small model on the chain-of-thought data generated by a large teacher model. The goal is to distill the reasoning ability of the larger model into the smaller model, allowing for improved performance in reasoning tasks. However, it is important to note that the effectiveness of this approach may vary depending on the specific task.","[11.1 New Pretraining Strategies\nAs investigated by Shin et al. (2022b), language\nmodel objectives are not equal to ICL abilities. Re-\nsearchers have proposed to bridge the gap between\npretraining objectives and ICL through interme-\ndiate tuning before inference (Section 4), which\nshows promising performance improvements. To\ntake it further, tailored pretraining objectives and\nmetrics for ICL have the potential to raise LLMs\nwith superior ICl capabilities.\n11.2 ICL Ability Distillation\nPrevious studies have shown that in-context learn-\ning for reasoning tasks emerges as the scale of\ncomputation and parameter exceed a certain thresh-\nold (Wei et al., 2022b). Transferring the ICL ability\nto smaller models could facilitate the model deploy-\nment greatly. Magister et al. (2022) showed that it\nis possible to distill the reasoning ability to small\nlanguage models such as T5-XXL. The distillationis achieved by finetuning the small model on the\nchain-of-thought data (Wei et al., 2022c) generated\nby a large teacher model. Although promising per-\nformance is achieved, the improvements are likely\ntask-dependent. Further investigation on improv-\n

In [8]:
from phoenix.session.evaluation import get_qa_with_reference

client = px.Client()
spans_dataframe = get_qa_with_reference(client)
spans_dataframe.head()

,input,output,reference
context.span_id,,,
bf7d55a697974b32beb362984e39e1b4,"What does the survey on advanced in-context learning techniques cover, including strategies for training and designing demonstrations?","The survey on advanced in-context learning techniques covers strategies for training and designing demonstrations. This includes training strategies, demonstration designing strategies, evaluation datasets and resources, as well as related analytical studies.","ment aims to improve the scalability and efficiency\nof ICL. As LMs continue to scale up, exploring\nways to effectively and efficiently utilize a larger\nnumber of demonstrations in ICL remains an ongo-\ning area of research.\n12 Conclusion\nIn this paper, we survey the existing ICL literature\nand provide an extensive review of advanced ICL\ntechniques, including training strategies, demon-\nstration designing strategies, evaluation datasets\nand resources, as well as related analytical studies.\nFurthermore, we highlight critical challenges and\npotential directions for future research. To the best\nof our knowledge, this is the first survey about ICL.\nWe hope this survey can highlight the current re-\nsearch status of ICL and shed light on future work\non this promising paradigm.\nReferences\nEkin Akyürek, Dale Schuurmans, Jacob An-\ndreas, Tengyu Ma, and Denny Zhou. 2022.\nWhat learning algorithm is in-context learn-\ning? investigations with linear models. CoRR ,\nabs/2211.15661.\nJean-Baptiste Alayrac, Jeff Donahue, Pauline Luc,\nAntoine Miech, Iain Barr, Yana Hasson, Karel\nLenc, Arthur Mensch, Katherine Millican, Mal-\ncolm Reynolds, et al. 2022. Flamingo: a vi-\nsual language model for few-shot learning. Ad-\nvances in Neural Information Processing Sys-\ntems, 35:23716–23736.\nShengnan An, Zeqi Lin, Qiang Fu, Bei Chen,\nNanning Zheng, Jian-Guang Lou, and Dong-\nmei Zhang. 2023. How do in-context exam-\nples affect compositional generalization? CoRR ,\nabs/2305.04835.\nAmir Bar, Yossi Gandelsman, Trevor Darrell,\nAmir Globerson, and Alexei Efros. 2022. Vi-\nsual prompting via image inpainting. Ad-\nvances in Neural Information Processing Sys-\ntems, 35:25005–25017.\nRichard Bellman. 1957. A markovian decision\nprocess. Journal of mathematics and mechanics ,\npages 679–684.\n\n(3) The perfor-\nmance advancement made by warmup encounters\na plateau when increasingly scaling up the training\ndata. This phenomenon appears both in supervised\nin-context training and self-supervised in-context\ntraining, indicating that LLMs only need a small\namount of data to adapt to learn from the context\nduring warmup.\n5 Demonstration Designing\nMany studies have shown that the performance\nof ICL strongly relies on the demonstration sur-\nface, including demonstration format, the order of\ndemonstration examples, and so on (Zhao et al.,\n2021; Lu et al., 2022). As demonstrations play a vi-\ntal role in ICL, in this section, we survey demonstra-\ntion designing strategies and classify them into two\ngroups: demonstration organization and demonstra-\ntion formatting, as shown in Table 1.\n5.1 Demonstration Organization\nGiven a pool of training examples, demonstration\norganization focuses on how to select a subset of\nexamples and the order of the selected examples.\n5.1.1 Demonstration Selection\nDemonstrations selection aims to answer a funda-\nmental question: Which examples are good exam-ples for ICL? We classify related studies into two\ncategories, including unsupervised methods based\non pre-defined metrics and supervised methods.\nUnsupervised Method Liu et al. (2022) showed\nthat selecting the closest neighbors as the in-context\nexamples is a good solution. The distance metrics\nare pre-defined L2 distance or cosine-similarity\ndistance based on sentence embeddings. They\nproposed KATE, a kNN-based unsupervised re-\ntriever for selecting in-context examples. In addi-\ntion to distance metrics, mutual information is also\na valuable selection metric (Sorens

In [9]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_correctness,
    context_recall,
    context_precision,
)

In [10]:
from phoenix.trace.langchain import OpenInferenceTracer

tracer = OpenInferenceTracer()

In [11]:
evaluation_result = evaluate(
    dataset=ragas_eval_dataset,
    metrics=[faithfulness, answer_correctness, context_recall, context_precision],
    callbacks=[tracer],
)
eval_scores_df = pd.DataFrame(evaluation_result.scores)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Invalid prediction format. Expected a list of dictionaries with keys 'TP', 'FP', 'FN'


In [23]:
eval_data_df = pd.DataFrame(evaluation_result.dataset)
assert eval_data_df.question.to_list() == list(
    reversed(spans_dataframe.input.to_list())
)
eval_scores_df.index = pd.Index(
    list(reversed(spans_dataframe.index.to_list())), name=spans_dataframe.index.name
)

In [26]:
from phoenix.trace import SpanEvaluations

for eval_name in eval_scores_df.columns:
    evals_df = eval_scores_df[[eval_name]].rename(columns={eval_name: "score"})
    evals = SpanEvaluations(eval_name, evals_df)
    px.log_evaluations(evals)

Sending Evaluations:   0%|          | 0/10 [00:00<?, ?it/s]

Sending Evaluations: 100%|██████████| 10/10 [00:00<00:00, 94.56it/s]


![](../../_static/imgs/arize-tracing2.gif)

## Embedding analysis
TBD:
- cluster queries
- color each data point based on question type?
- display average score for each cluster